# Interoperability between XProj and 3rd-party Xarray geospatial extensions

3rd-party geospatial extensions may leverage XProj in different ways:

- simply consume the API exposed via the `.proj` Dataset / DataArray accessor
- register a custom Xarray Dataset / DataArray accessor that also inherits from `xproj.ProjAccessorMixin` (example below)
- implement one or more methods of `xproj.ProjIndexMixin` in custom Xarray indexes  (example below)

In [1]:
import pyproj
import xarray as xr
import xproj

xr.set_options(display_expand_indexes=True);

## CRS-aware Xarray accessor

Here below is a basic example of a custom Xarray Dataset accessor that is also explictly registered as a "geo" accessor via the `xproj.register_accessor` class decorator. Important note: the latter must be applied after (on top of) the Xarray register decorators.

Registering this "geo" accessor allows executing custom logic from within the accessor (via the CRS interface) when calling `xproj` API.

In [2]:
@xproj.register_accessor
@xr.register_dataset_accessor("geo")
class GeoAccessor(xproj.ProjAccessorMixin):

    def __init__(self, obj):
        self._obj = obj

    @property
    def crs(self):
        # Just reusing XProj's API
        # (Assuming this accessor only supports single-CRS datasets)
        return self._obj.proj.crs

    def _proj_set_crs(self, crs_coord_name, crs):
        # Nothing much done here, just printing something before
        # returning the Xarray dataset unchanged

        print(f"from GeoAccessor: new CRS of {crs_coord_name!r} is {crs}!")
        return self._obj


Let's see if it works as expected.

In [3]:
# create an empty dataset, The `.geo.crs` property is uninitialized

ds = xr.Dataset()

ds.geo.crs is None

True

In [4]:
# initialize the CRS via `.proj.assign_crs()`

ds_wgs84 = ds.proj.assign_crs(spatial_ref=pyproj.CRS.from_user_input("epsg:4326"))

from GeoAccessor: new CRS of 'spatial_ref' is epsg:4326!


In [5]:
ds_wgs84

<xarray.Dataset> Size: 8B
Dimensions:      ()
Coordinates:
  * spatial_ref  int64 8B 0
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [6]:
# Access CRS via the `.geo` accessor

ds_wgs84.geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## CRS-aware Xarray index

Here below is a basic example of a custom Xarray index that adds some CRS-aware functionality on top of Xarray's default `PandasIndex`.

In [7]:
import warnings


class GeoIndex(xr.indexes.PandasIndex, xproj.ProjIndexMixin):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._crs = None

    def sel(self, *args, **kwargs):
        if self._crs is not None:
            warnings.warn(f"make sure that indexer labels have CRS {self._crs}!", UserWarning)
        
        return super().sel(*args, **kwargs)

    def _proj_get_crs(self):
        return self._crs
    
    def _proj_set_crs(self, crs_coord_name, crs):
        # `crs_coord_name` not used here (assuming single-CRS dataset)

        print(f"set CRS of index {self!r} to crs={crs}!")

        self._crs = crs
        return self

    def _copy(self, deep=True, memo=None):
        # bug in PandasIndex? crs attribute not copied here
        obj = super()._copy(deep=deep, memo=memo)
        obj._crs = self._crs
        return obj

    def _repr_inline_(self, max_width=70):
        return f"{type(self).__name__} (crs={self._crs})"

    def __repr__(self):
        return f"{type(self).__name__} (crs={self._crs})"


Let's see how it works

In [8]:
# Create a new Dataset with a latitude coordinate and a (default) PandasIndex

ds = xr.Dataset({"lat": [1, 2, 3]})
ds

<xarray.Dataset> Size: 24B
Dimensions:  (lat: 3)
Coordinates:
  * lat      (lat) int64 24B 1 2 3
Data variables:
    *empty*

In [9]:
# Replace the PandasIndex with a GeoIndex (crs not yet initialized)

ds_geo = ds.drop_indexes(["lat"]).set_xindex("lat", GeoIndex)
ds_geo

<xarray.Dataset> Size: 24B
Dimensions:  (lat: 3)
Coordinates:
  * lat      (lat) int64 24B 1 2 3
Data variables:
    *empty*

In [10]:
# Initialize the CRS via `.proj.assign_crs()`

temp = ds_geo.proj.assign_crs(spatial_ref="epsg:4326")
temp

from GeoAccessor: new CRS of 'spatial_ref' is epsg:4326!


<xarray.Dataset> Size: 32B
Dimensions:      (lat: 3)
Coordinates:
  * lat          (lat) int64 24B 1 2 3
  * spatial_ref  int64 8B 0
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [11]:
# Must explicitly map the spatial reference coordinate
# to the coordinate with a GeoIndex

ds_geo_wgs84 = temp.proj.map_crs(spatial_ref=["lat"])

set CRS of index GeoIndex (crs=None) to crs=epsg:4326!


In [12]:
# The index of the `lat` coordinate now has its CRS initialized!

ds_geo_wgs84

<xarray.Dataset> Size: 32B
Dimensions:      (lat: 3)
Coordinates:
  * spatial_ref  int64 8B 0
  * lat          (lat) int64 24B 1 2 3
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [13]:
# "CRS-aware" data selection (just a warning emitted here)

ds_geo_wgs84.sel(lat=1)

/var/folders/xd/3ls911kd6_n2wphwwd74b1dc0000gn/T/ipykernel_49494/2407704525.py:12: UserWarning: make sure that indexer labels have CRS epsg:4326!
  warnings.warn(f"make sure that indexer labels have CRS {self._crs}!", UserWarning)


<xarray.Dataset> Size: 16B
Dimensions:      ()
Coordinates:
  * spatial_ref  int64 8B 0
    lat          int64 8B 1
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

Note: since `GeoIndex` implements the `__proj_get_crs__` interface it is also possible to get the CRS from the "lat" coordinate like so:

In [14]:
ds_geo_wgs84.proj("lat").crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Caveat

Changing the CRS via `.proj.assign_crs()` requires to manually call `.proj.map_crs()` again in order to synchronize the new CRS with the coordinate index(es).

In [15]:
# Change CRS of "spatial_ref" (no effect on the GeoIndex of the "lat" coordinate!)

temp = ds_geo_wgs84.proj.assign_crs(spatial_ref="epsg:32662", allow_override=True)
temp

from GeoAccessor: new CRS of 'spatial_ref' is epsg:32662!


<xarray.Dataset> Size: 32B
Dimensions:      (lat: 3)
Coordinates:
  * spatial_ref  int64 8B 0
  * lat          (lat) int64 24B 1 2 3
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:32662)

In [16]:
# Getting a unique CRS for the Dataset returns an error!

temp.proj.crs

ValueError: found multiple CRS in Dataset or DataArray:
spatial_ref: EPSG:32662
lat: EPSG:4326

In [17]:
# re-map "spatial_ref" to "lat"

ds_geo_plate_carree = temp.proj.map_crs(spatial_ref=["lat"])
ds_geo_plate_carree

set CRS of index GeoIndex (crs=epsg:4326) to crs=epsg:32662!


<xarray.Dataset> Size: 32B
Dimensions:      (lat: 3)
Coordinates:
  * spatial_ref  int64 8B 0
  * lat          (lat) int64 24B 1 2 3
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:32662)

In [18]:
# The latter Dataset now has a unique CRS

ds_geo_plate_carree.proj.crs

<Projected CRS: EPSG:32662>
Name: WGS 84 / Plate Carree
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Coordinate Operation:
- name: World Equidistant Cylindrical (Sphere)
- method: Equidistant Cylindrical (Spherical)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich